In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import utils 
utils.init_logging()

import polars as pl
from polars import col, lit
import altair as alt
import simrs
import input_builder as b
import scenarios
import cli
import analysis

In [3]:
scenario = scenarios.barbell_scenario(ticks=10, num_agents=10)
(actions, agents, markets, events) = scenarios.run_scenario(scenario)

In [4]:
print("agents") 
agents.head()

agents


behavior,cargo,coins,id,pos,tick
str,str,f64,str,str,i64
"""Exhaustive""",null,1000.0,"""a8""","""la""",0
"""Exhaustive""",null,1000.0,"""a7""","""la""",0
"""Exhaustive""",null,1000.0,"""a4""","""la""",0
"""Exhaustive""",null,1000.0,"""a6""","""la""",0
"""Exhaustive""",null,1000.0,"""a9""","""la""",0


In [5]:
print("markets") 
markets.head()

markets


consumption,good,port,price,production,supply,tick
f64,str,str,f64,f64,f64,i64
100.0,"""Wheat""","""ra""",98.039216,100.0,1020.0,0
100.0,"""Wheat""","""m1""",102.040816,100.0,980.0,0
100.0,"""Wheat""","""m2""",100.0,100.0,1000.0,0
100.0,"""Wheat""","""la""",102.040816,100.0,980.0,0
100.0,"""Wheat""","""lb""",102.040816,100.0,980.0,0


In [6]:
print("actions")
actions.head(10)

actions


action,agent_id,port_id,tick,good
str,str,str,i64,str
"""Move""","""a8""","""m1""",0,null
"""Move""","""a7""","""m1""",0,null
"""Move""","""a4""","""m1""",0,null
"""Move""","""a6""","""m1""",0,null
"""Move""","""a9""","""m1""",0,null
"""Move""","""a5""","""m1""",0,null
"""Move""","""a3""","""m1""",0,null
"""Move""","""a10""","""m1""",0,null
"""Move""","""a2""","""m1""",0,null


In [7]:
events.head(10)

agent,amt,cost,event,good,port,tick
str,i64,f64,str,str,str,i64
"""a8""",1,100.05005,"""Trade""","""Wheat""","""m2""",2
"""a7""",1,100.15025,"""Trade""","""Wheat""","""m2""",2
"""a4""",1,100.250652,"""Trade""","""Wheat""","""m2""",2
"""a6""",1,100.351255,"""Trade""","""Wheat""","""m2""",2
"""a9""",1,100.452059,"""Trade""","""Wheat""","""m2""",2
"""a5""",1,100.553067,"""Trade""","""Wheat""","""m2""",2
"""a3""",1,100.654278,"""Trade""","""Wheat""","""m2""",2
"""a10""",1,100.755693,"""Trade""","""Wheat""","""m2""",2
"""a2""",1,100.857313,"""Trade""","""Wheat""","""m2""",2


In [8]:
analysis.plot_agents(agents)

alt.LayerChart(...)

In [9]:
analysis.plot_agent_locations(agents)

alt.Chart(...)

In [10]:
analysis.plot_prices_by_port(markets)

alt.LayerChart(...)

In [11]:
analysis.plot_prices_by_port(analysis.no_agent_markets(scenario)) 

alt.LayerChart(...)

In [12]:
routes = analysis.make_routes(events)
routes

agent,amt,buy_cost,src,buy_tick,sell_cost,dst,sell_tick,profit,buy_price,sell_price
str,i64,f64,str,i64,f64,str,i64,f64,f64,f64
"""a9""",1,100.452059,"""m2""",2,-99.552041,"""m3""",3,-0.900019,100.452059,99.552041
"""a9""",1,99.453033,"""m3""",4,-100.553067,"""m2""",5,1.100034,99.453033,100.553067
"""a9""",1,100.351255,"""m2""",6,-99.651245,"""m3""",7,-0.700009,100.351255,99.651245
"""a9""",1,99.552041,"""m3""",8,-100.452059,"""m2""",9,0.900019,99.552041,100.452059
"""a3""",1,100.654278,"""m2""",2,-99.354222,"""m3""",3,-1.300056,100.654278,99.354222
"""a3""",1,99.651245,"""m3""",4,-100.351255,"""m2""",5,0.700009,99.651245,100.351255
"""a3""",1,100.553067,"""m2""",6,-99.453033,"""m3""",7,-1.100034,100.553067,99.453033
"""a3""",1,99.750648,"""m3""",8,-100.250652,"""m2""",9,0.500004,99.750648,100.250652
"""a8""",1,100.05005,"""m2""",2,-99.95005,"""m3""",3,-0.1,100.05005,99.95005


In [13]:
blag = routes.with_columns(pl.struct("src", "dst").alias("route"))
blag.groupby("route").agg(pl.count()).sort("count", descending=True)

route,count
struct[2],u32
"{""m2"",""m3""}",18
"{""m3"",""m2""}",18
"{""ra"",""m3""}",2


In [14]:
analysis.plot_trades(routes)

alt.Chart(...)

In [15]:
analysis.plot_buy_and_sell_prices(routes) + analysis.plot_trades(routes)

alt.LayerChart(...)

## Port level analysis
- How much does each port trade?
- Trade volume bucketted 
- Volume in dollars 
- Biggest trading partners (other ports)

In [16]:
## Total goods traded per port
events.groupby("port").agg(pl.sum("amt"))
# def plot_

port,amt
str,i64
"""m3""",-2
"""m2""",0
"""ra""",2


Metrics for individual agent
- total coins
- coins per tick
- died?

Metrics for population of agents
- min, max, median, mean, std of agent coins

How well did the agents equalize prices?
- box plot of prices
- stddev of pricesf

Construct 'trades'
- bought Cargo at StartPort for BuyPrice
- sold Cargo at EndPort for SellPrice
- profit = SellPrice - BuyPrice
- Route = StartPort -> .. ->  EndPort
- RouteLength = len(Route)